# Gridding a synthetic ground survey

Let's perform a synthetic ground survey of the synthetic model made out of prisms and try to interpolate the observed data on a regular grid.
Because the model is synthetic, we can compute the true value of the field on this regular grid. Therefore, we have an objective way to score the interpolation. This allow us to objectively compare the different source layouts.

Firstly, we want to import useful packages and define some minor functions that will help us to visualize better the notebook cells and reduce ammount of code on them.

**Table of Contents**

- [1. Define parameters used on the gridding](#1.-Define-parameters-used-on-the-gridding)
- [2. Create the synthetic ground survey](#2.-Create-the-synthetic-ground-survey)
- [3. Compute the field of the synthetic model on a grid](#3.-Compute-the-field-of-the-synthetic-model-on-a-grid)
- [4. Grid data with constant depth](#4.-Grid-data-with-constant-depth)
    - [4.1. One source beneath each data point](#4.1.-One-source-beneath-each-data-point)
    - [4.2. Source beneath block reduced observation points](#4.2.-Source-beneath-block-reduced-observation-points)
    - [4.3. Regular grid of source points](#4.3.-Regular-grid-of-source-points)
- [5. Grid data with constant depth](#5.-Grid-data-with-relative-depth)
    - [5.1. One source beneath each data point](#5.1.-One-source-beneath-each-data-point)
    - [5.2. Source beneath block reduced observation points](#5.2.-Source-beneath-block-reduced-observation-points)
- [6. Grid data with variable relative depth](#6.-Grid-data-with-variable-relative-depth)
    - [6.1. One source beneath each data point](#6.1.-One-source-beneath-each-data-point)
    - [6.2. Source beneath block reduced observation points](#6.2.-Source-beneath-block-reduced-observation-points)
- [7. Plot and compare the best predictions](#7.-Plot-and-compare-the-best-predictions)

**Import useful packages**

In [ ]:
from IPython.display import display, HTML
import itertools
import pyproj
import numpy as np
import pandas as pd
import verde as vd
import harmonica as hm
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection

from eql_source_layouts import (
    synthetic_model,
    source_bellow_data,
    block_reduced_sources,
    grid_sources,
)

**Define functions to plot gridding results**

In [ ]:
def plot_prediction(
    grid,
    prediction,
    target,
    title="Prediction",
    figsize=(12, 6),
    shrink_cbar=0.7,
):
    """
    Plot prediction and comparison with target
    """
    fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=figsize)
    tmp = ax1.pcolormesh(*grid[:2], prediction)
    plt.colorbar(tmp, ax=ax1, shrink=shrink_cbar, label=field_units)
    ax1.set_aspect("equal")
    ax1.set_title(title)

    maxabs = vd.maxabs(target - prediction)
    tmp = ax2.pcolormesh(
        *grid[:2],
        target - best_prediction,
        vmin=-maxabs,
        vmax=maxabs,
        cmap="seismic",
    )
    plt.colorbar(tmp, ax=ax2, shrink=shrink_cbar, label=field_units)
    ax2.set_aspect("equal")
    ax2.set_title("Difference between target and prediction")
    plt.tight_layout()
    plt.show()
    
    
def histogram_errors(prediction, target, bins=20):
    """
    Plot histogram of prediction errors
    """
    plt.hist((target - prediction).ravel(), bins=bins)
    plt.title("Histogram of prediction errors")
    plt.show()

**Define function to print prediction parameters and scores with style**

In [ ]:
def print_params_scores(parameters_sets, scores):
    """
    Display parameters and scores with style
    
    It also highlights the row that contains the maximum score
    """
    df = {}
    for keys in parameters_sets[0]:
        df[keys] = []
    df["score"] = []
    for parameters, score in zip(parameters_sets, scores):
        for key, param in parameters.items():
            df[key].append(param)
        df["score"].append(score)
    df = pd.DataFrame(df)
    display(df.style.apply(highlight_max_row, column="score", axis=None))
    return df


def highlight_max_row(df, column, color="orange"):
    """
    Highlight the row that contains the max value of a column
    """
    style = df.copy()
    is_max = (df[column] == df[column].max())
    background = [
        'background-color: {}'.format(color) if i else '' for i in is_max
    ]
    for col in style:
        style[col] = background
    return style

## 1. Define parameters used on the gridding

Let's define all the parameters that will be used on this notebook in the following cells. These control the results that will be obtain on the rest of the notebook. If you want to change something (like the computation height of the grid, the survey region, interpolation parameters, etc), you can just do it here.

We will avoid hardcoding parameters outside of these few cells in order to facilitate reproducibility and keep things more organized.

In [ ]:
# Define a survey region of 1 x 1 degrees (~ 100km x 100km)
region_degrees = (-0.5, 0.5, -0.5, 0.5)

# Define bottom and top of the synthetic model
model_bottom, model_top = -10e3, 0

# Define which field will be meassured
field = "g_z"
field_units = "mGal"

# Define standard deviation for the Gaussian noise that
# will be added to the synthetic survey (in mGal)
noise_std = 1

# Define a seed to reproduce the same results on each run 
np.random.seed(12345)


# Define the spacing of the target regular grid
# and its observation height
grid_spacing = 2e3
grid_height = 2000

# Define set of interpolation parameters
# ======================================
# Define dampings used on every fitting of the gridder
dampings = [None, 1e-4, 1e-3, 1e-2]
# Define values of constant depth
constant_depths = [1e3, 2e3, 5e3, 10e3, 15e3]
# Define values of relative depth
relative_depths = [1e3, 2e3, 5e3, 10e3, 15e3]
# Define parameters for the grid layout:
#    spacing, depth and padding
source_grid_spacings = [0.5e3, 1e3, 2e3]
source_grid_depths = [1e3, 2e3, 5e3]
source_grid_paddings = [0, 0.1, 0.2]
# Define parameters for variable relative depth layouts:
#    depth factor, depth shift and k_values
depth_factors = [0.5, 1, 5, 10]
depth_shifts = [-100, -1000, -5000]
k_values = [1, 10]
# We will set the block spacing for the block reduced
# layouts equal to the target grid spacing
block_spacing = grid_spacing

# Finally, define a dict where the best predictions will be
# stored to plot them altogether at the end of the notebook.
best_predictions = {
    "constant_depth": {},
    "relative_depth": {},
    "variable_relative_depth": {},
}

## 2. Create the synthetic ground survey

To create the ground survey we need to load the synthetic model made out of prisms, the location of the observation points and then compute the field that the prisms generate on those points.

Get coordinates of observation points

In [ ]:
survey = hm.synthetic.ground_survey(region=region_degrees)
display(survey)

Project survey points into Cartesian coordinates

In [ ]:
projection = pyproj.Proj(proj="merc", lat_ts=0)
survey["easting"], survey["northing"] = projection(
    survey.longitude.values, survey.latitude.values
)
display(survey)

# Define region boundaries in projected coordinates
region = (
    survey.easting.values.min(),
    survey.easting.values.max(),
    survey.northing.min(),
    survey.northing.max(),
)

Plot the survey points

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
tmp = ax.scatter(survey.easting, survey.northing, c=survey.height, s=6)
plt.colorbar(tmp, ax=ax, label="m")
ax.set_aspect("equal")
ax.set_title("Height of ground survey points")
plt.show()

Get the synthetic model

In [ ]:
# Define model region
model_region = tuple(list(region) + [model_bottom, model_top])

# Create synthetic model
model = synthetic_model(model_region)
print(model.keys())
print("Number of prisms: {}".format(len(model["densities"])))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.add_collection(PatchCollection(model["rectangles"], match_original=True))
ax.set_aspect("equal")
ax.set_title("Synthetic model made out of prisms")
ax.set_xlim(region[:2])
ax.set_ylim(region[2:4])
plt.show()

Compute the gravity field (g_z) on the observation points in mGal and add Gaussian noise

In [ ]:
coordinates = (survey.easting, survey.northing, survey.height)
survey[field] = hm.prism_gravity(
    coordinates, model["prisms"], model["densities"], field=field
) + np.random.normal(scale=noise_std, size=survey.easting.size)
display(survey)

Plot the observed field

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
tmp = ax.scatter(survey.easting, survey.northing, c=getattr(survey, field), s=6)
plt.colorbar(tmp, ax=ax, label=field_units)
ax.set_aspect("equal")
ax.set_title("Synthetic ground survey")
plt.show()

## 3. Compute the field of the synthetic model on a grid

Now, let's compute the gravity field that the synthetic model generates on the regular grid. These results will serve as a target for the interpolations using different source layouts.

Build the regular grid

In [ ]:
grid = vd.grid_coordinates(
    region=region, spacing=grid_spacing, extra_coords=grid_height
)

Compute the synthetic gravity field on the grid

In [ ]:
target = hm.prism_gravity(
    grid, model["prisms"], model["densities"], field=field
)

Plot target gravity field

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
tmp = ax.pcolormesh(*grid[:2], target)
plt.colorbar(tmp, ax=ax, shrink=0.7, label=field_units)
ax.set_aspect("equal")
ax.set_title("Target grid values")
plt.show()

## 4. Grid data with constant depth

Now, let's grid the synthetic data through different source layouts, but keeping the constant depth approach.
We will compare each interpolation with the target data in order to assess for the quality of each source layout.

Because the gridder could take several parameters that control the interpolation, we will perform several predictions with different set of parameters and then score each one of them against the target data. We will choose the best prediction as the one that produces the highest score. Because this score is computed agains *true data*, it's an objective quality estimator of the prediction.

### 4.1. One source beneath each data point

Let's perform several interpolations on the grid by setting a single source point beneath each data point at a constant depth. This can be achieved throught `harmonica.EQLHarmonic` by passing a points layout build through the `source_bellow_data()` function.
Each interpolation is performed with different values of the `damping` and `constant_depth` parameters. Then we score the prediction against the target field and store all these scores.

Define the set of parameters that will be used on the interpolations

In [ ]:
parameters_sets = [
    dict(damping=combo[0], constant_depth=combo[1])
    for combo in itertools.product(dampings, constant_depths)
]

Perform the interpolations and score them against the target data

In [ ]:
scores = []
for parameters in parameters_sets:
    # Create the source points layout by putting one source beneath
    # each data point
    points = source_bellow_data(
        coordinates, depth_type="constant_depth", **parameters
    )
    # Initialize the gridder passing the points and the damping
    eql = hm.EQLHarmonic(points=points, damping=parameters["damping"])
    # Fit the gridder giving the survey data
    eql.fit(coordinates, getattr(survey, field))
    # Predict the field on the regular grid
    prediction = eql.predict(grid)
    # Score the prediction against target data
    scores.append(r2_score(target, prediction))

Let's see all the computed scores. The highlighted row indicates the set of parameters that produces the maximum score.

In [ ]:
df = print_params_scores(parameters_sets, scores)

Keep the best prediction and plot it

In [ ]:
best = np.argmax(scores)
best_params = parameters_sets[best]
print("Best score:", scores[best])
print("Best parameters:", best_params)

points = source_bellow_data(
    coordinates, depth_type="constant_depth", **best_params
)
eql = hm.EQLHarmonic(points=points, damping=best_params["damping"])
eql.fit(coordinates, getattr(survey, field))
best_prediction = eql.predict(grid)

print("Number of point sources:", points[0].size)

# Store the best prediction
best_predictions["constant_depth"]["source_beneath_data"] = best_prediction

plot_prediction(grid, best_prediction, target)
histogram_errors(best_prediction, target)

### 4.2. Source beneath block reduced observation points

Now we will do the same, but with a different source layout: block reduce the observation points and put one source beneath each reduced coordinate at a constant depth.
For that we will need to make use of our `block_reduced_sources()` function.
Each interpolation will be performed with different values of the `damping` and `constant_depth` parameters. The `spacing` parameter will be set equal to the target grid spacing. Then we score the prediction against the target field and store all these scores.

Define the set of parameters that will be used on the interpolations

In [ ]:
parameters_sets = [
    dict(
        damping=combo[0],
        constant_depth=combo[1],
        spacing=block_spacing,
    )
    for combo in itertools.product(dampings, constant_depths)
]

Perform the interpolations and score them against the target data

In [ ]:
scores = []
for parameters in parameters_sets:
    # Create the custom source points layout by block reducing the
    # observed coordinates and setting the point sources at a 
    # constant depth beneath them.
    points = block_reduced_sources(
        coordinates, depth_type="constant_depth", **parameters
    )
    # Initialize the gridder passing the points layout and the damping
    eql = hm.EQLHarmonic(points=points, damping=parameters["damping"])
    # Fit the gridder giving the survey data
    eql.fit(coordinates, getattr(survey, field))
    # Predict the field on the regular grid
    prediction = eql.predict(grid)
    # Score the prediction against target data
    scores.append(r2_score(target, prediction))

Let's see all the computed scores. The highlighted row indicates the set of parameters that produces the maximum score.

In [ ]:
df = print_params_scores(parameters_sets, scores)

Keep the best prediction and plot it

In [ ]:
best = np.argmax(scores)
best_params = parameters_sets[best]
print("Best score:", scores[best])
print("Best parameters:", best_params)

points = block_reduced_sources(
    coordinates, depth_type="constant_depth", **best_params
)
eql = hm.EQLHarmonic(points=points, damping=best_params["damping"])
eql.fit(coordinates, getattr(survey, field))
best_prediction = eql.predict(grid)

print("Number of point sources:", points[0].size)

# Store the best prediction
best_predictions["constant_depth"]["block_reduced"] = best_prediction

plot_prediction(grid, best_prediction, target)
histogram_errors(best_prediction, target)

### 4.3. Regular grid of source points

Finally, let's build a regular grid of source points located at a constant depth beneath the mean height of observation points.
Each interpolation will be performed with different values of the `damping` and `constant_depth`. The `spacing` parameter will be set equal to the target grid spacing. Then we score the prediction against the target field and store all these scores.

Define the set of parameters that will be used on the interpolations

In [ ]:
parameters_sets = [
    dict(
        damping=combo[0],
        constant_depth=combo[1],
        pad=combo[2],
        spacing=combo[3],
    )
    for combo in itertools.product(
        dampings,
        source_grid_depths,
        source_grid_paddings,
        source_grid_spacings,
    )
]

Perform the interpolations and score them against the target data. When fitting the gridder, it will raise a warning due to the fact that the determination of the coefficients for each source point is an under-determined problem: the number of point sources is bigger than the number of observation points.

In [ ]:
# Compute mean height of observation points
mean_height = survey.height.mean()

scores = []
for parameters in parameters_sets:
    # Create the custom source points layout by putting them on a
    # regular grid at a constant depth beneath the data points.
    points = grid_sources(
        coordinates, depth_type="constant_depth", **parameters
    )
    # Initialize the gridder passing the points layout and the damping
    eql = hm.EQLHarmonic(points=points, damping=parameters["damping"])
    # Fit the gridder giving the survey data
    eql.fit(coordinates, getattr(survey, field))
    # Predict the field on the regular grid
    prediction = eql.predict(grid)
    # Score the prediction against target data
    scores.append(r2_score(target, prediction))

Let's see all the computed scores. The highlighted row indicates the set of parameters that produces the maximum score.

In [ ]:
df = print_params_scores(parameters_sets, scores)

Keep the best prediction and plot it

In [ ]:
best = np.argmax(scores)
best_params = parameters_sets[best]
print("Best score:", scores[best])
print("Best parameters:", best_params)

points = grid_sources(
    coordinates, depth_type="constant_depth", **best_params
)
eql = hm.EQLHarmonic(points=points, damping=best_params["damping"])
eql.fit(coordinates, getattr(survey, field))
best_prediction = eql.predict(grid)

print("Number of point sources:", points[0].size)

# Store the best prediction
best_predictions["constant_depth"]["grid"] = best_prediction

plot_prediction(grid, best_prediction, target)
histogram_errors(best_prediction, target)

## 5. Grid data with relative depth

Now, let's grid the synthetic data through different source layouts, but keeping the relative depth approach.
We will compare each interpolation with the target data in order to assess for the quality of each source layout.

Because the gridder could take several parameters that control the interpolation, we will perform several predictions with different set of parameters and then score each one of them against the target data. We will choose the best prediction as the one that produces the highest score. Because this score is computed agains *true data*, it's an objective quality estimator of the prediction.

### 5.1. One source beneath each data point

Let's perform several interpolations on the grid by setting a single source point beneath each data point at a relative depth. This can be achieved throught `harmonica.EQLHarmonic` by passing a points layout build through the `source_bellow_data()` function.
Each interpolation is performed with different values of the `damping` and `relative_depth` parameters. Then we score the prediction against the target field and store all these scores.

Define the set of parameters that will be used on the interpolations

In [ ]:
parameters_sets = [
    dict(
        damping=combo[0],
        relative_depth=combo[1],
    )
    for combo in itertools.product(dampings, relative_depths)
]

Perform the interpolations and score them against the target data

In [ ]:
scores = []
for parameters in parameters_sets:
    # Create the source points layout by putting one source beneath
    # each data point
    points = source_bellow_data(
        coordinates, depth_type="relative_depth", **parameters
    )
    # Initialize the gridder passing the points and the damping
    eql = hm.EQLHarmonic(points=points, damping=parameters["damping"])
    # Fit the gridder giving the survey data
    eql.fit(coordinates, getattr(survey, field))
    # Predict the field on the regular grid
    prediction = eql.predict(grid)
    # Score the prediction against target data
    scores.append(r2_score(target, prediction))

Let's see all the computed scores. The highlighted row indicates the set of parameters that produces the maximum score.

In [ ]:
df = print_params_scores(parameters_sets, scores)

Keep the best prediction and plot it

In [ ]:
best = np.argmax(scores)
best_params = parameters_sets[best]
print("Best score:", scores[best])
print("Best parameters:", best_params)

points = source_bellow_data(
    coordinates, depth_type="relative_depth", **best_params
)
eql = hm.EQLHarmonic(points=points, damping=best_params["damping"])
eql.fit(coordinates, getattr(survey, field))
best_prediction = eql.predict(grid)

print("Number of point sources:", points[0].size)

# Store the best prediction
best_predictions["relative_depth"]["source_beneath_data"] = best_prediction

plot_prediction(grid, best_prediction, target)
histogram_errors(best_prediction, target)

### 5.2. Source beneath block reduced observation points

Now we will do the same, but with a different source layout: block reduce the observation points and put one source beneath each reduced coordinate at a relative depth.
For that we will need to make use of our `block_reduced_sources()` function.
Each interpolation will be performed with different values of the `damping` and `relative_depth` parameters. The `spacing` parameter will be set equal to the target grid spacing. Then we score the prediction against the target field and store all these scores.

Define the set of parameters that will be used on the interpolations

In [ ]:
parameters_sets = [
    dict(
        damping=combo[0],
        relative_depth=combo[1],
        spacing=block_spacing,
    )
    for combo in itertools.product(dampings, relative_depths)
]

Perform the interpolations and score them against the target data

In [ ]:
scores = []
for parameters in parameters_sets:
    # Create the custom source points layout by block reducing the
    # observed coordinates and setting the point sources at a
    # relative depth beneath them.
    points = block_reduced_sources(
        coordinates, depth_type="relative_depth", **parameters
    )
    # Initialize the gridder passing the points layout and the damping
    eql = hm.EQLHarmonic(points=points, damping=parameters["damping"])
    # Fit the gridder giving the survey data
    eql.fit(coordinates, getattr(survey, field))
    # Predict the field on the regular grid
    prediction = eql.predict(grid)
    # Score the prediction against target data
    scores.append(r2_score(target, prediction))

Let's see all the computed scores. The highlighted row indicates the set of parameters that produces the maximum score.

In [ ]:
df = print_params_scores(parameters_sets, scores)

Keep the best prediction and plot it

In [ ]:
best = np.argmax(scores)
best_params = parameters_sets[best]
print("Best score:", scores[best])
print("Best parameters:", best_params)

points = block_reduced_sources(
    coordinates, depth_type="relative_depth", **best_params
)
eql = hm.EQLHarmonic(points=points, damping=best_params["damping"])
eql.fit(coordinates, getattr(survey, field))
best_prediction = eql.predict(grid)

print("Number of point sources:", points[0].size)

# Store the best prediction
best_predictions["relative_depth"]["block_reduced"] = best_prediction

plot_prediction(grid, best_prediction, target)
histogram_errors(best_prediction, target)

## 6. Grid data with variable relative depth

We will perform the same steps as before, but now with variable relative depth.
For every source layout we will use our `variable_relative_depth()` function to compute the depth of each source point.

### 6.1. One source beneath each data point

Let's perform several interpolations on the grid by setting a single source point beneath each data point at a variable relative depth.
Each interpolation will be performed with different values of the `damping`, `depth_factor`, `depth_shift` and`k_nearest` parameters. Then we score the prediction against the target field and store all these scores.

Define the set of parameters that will be used on the interpolations

In [ ]:
parameters_sets = [
    dict(
        damping=combo[0],
        depth_factor=combo[1],
        depth_shift=combo[2],
        k_nearest=combo[3],
    )
    for combo in itertools.product(
        dampings, depth_factors, depth_shifts, k_values
    )
]

Perform the interpolations and score them against the target data

In [ ]:
scores = []
for parameters in parameters_sets:
    # Create the source points layout by putting one source beneath
    # each data point.
    points = source_bellow_data(
        coordinates, depth_type="variable_relative_depth", **parameters
    )
    # Initialize the gridder passing the points and the damping
    eql = hm.EQLHarmonic(points=points, damping=parameters["damping"])
    # Fit the gridder giving the survey data
    eql.fit(coordinates, getattr(survey, field))
    # Predict the field on the regular grid
    prediction = eql.predict(grid)
    # Score the prediction against target data
    scores.append(r2_score(target, prediction))

Let's see all the computed scores. The highlighted row indicates the set of parameters that produces the maximum score.

In [ ]:
df = print_params_scores(parameters_sets, scores)

Keep the best prediction and plot it

In [ ]:
best = np.argmax(scores)
best_params = parameters_sets[best]
print("Best score:", scores[best])
print("Best parameters:", best_params)

points = source_bellow_data(
    coordinates, depth_type="variable_relative_depth", **best_params
)
eql = hm.EQLHarmonic(points=points, damping=best_params["damping"])
eql.fit(coordinates, getattr(survey, field))
best_prediction = eql.predict(grid)

print("Number of point sources:", points[0].size)

# Store the best prediction
best_predictions["variable_relative_depth"]["source_beneath_data"] = best_prediction

plot_prediction(grid, best_prediction, target)
histogram_errors(best_prediction, target)

### 6.2. Source beneath block reduced observation points

Now we will do the same, but with a different source layout: block reduce the observation points and put one source beneath each reduced coordinate at a variable relative depth.
For that we will need to make use of our `block_reduce_points()` function.
Each interpolation will be performed with different values of the `damping`, `depth_factor`, `depth_shift` and `k_nearest` parameters. The `spacing` parameter will be set equal to the target grid spacing. Then we score the prediction against the target field and store all these scores.

Define the set of parameters that will be used on the interpolations

In [ ]:
parameters_sets = [
    dict(
        damping=combo[0],
        spacing=block_spacing,
        depth_factor=combo[1],
        depth_shift=combo[2],
        k_nearest=combo[3],
    )
    for combo in itertools.product(
        dampings, depth_factors, depth_shifts, k_values
    )
]

Perform the interpolations and score them against the target data

In [ ]:
scores = []
for parameters in parameters_sets:
    # Create the custom source points layout by block reducing the
    # observed coordinates and setting the point sources at a
    # relative depth beneath them.
    points = block_reduced_sources(
        coordinates, depth_type="variable_relative_depth", **parameters
    )
    # Initialize the gridder passing the points layout and the damping
    eql = hm.EQLHarmonic(points=points, damping=parameters["damping"])
    # Fit the gridder giving the survey data
    eql.fit(coordinates, getattr(survey, field))
    # Predict the field on the regular grid
    prediction = eql.predict(grid)
    # Score the prediction against target data
    scores.append(r2_score(target, prediction))

Let's see all the computed scores. The highlighted row indicates the set of parameters that produces the maximum score.

In [ ]:
df = print_params_scores(parameters_sets, scores)

Keep the best prediction and plot it

In [ ]:
best = np.argmax(scores)
best_params = parameters_sets[best]
print("Best score:", scores[best])
print("Best parameters:", best_params)

points = block_reduced_sources(
    coordinates, depth_type="variable_relative_depth", **best_params
)
eql = hm.EQLHarmonic(points=points, damping=best_params["damping"])
eql.fit(coordinates, getattr(survey, field))
best_prediction = eql.predict(grid)

print("Number of point sources:", points[0].size)

# Store the best prediction
best_predictions["variable_relative_depth"]["block_reduced"] = best_prediction

plot_prediction(grid, best_prediction, target)
histogram_errors(best_prediction, target)

## 7. Plot and compare the best predictions

In [ ]:
# Get max absolute value of the difference between target an all predictions
# in order to plot every difference with the same color scale.
maxabs = vd.maxabs(
    tuple(
        target - i
        for subset in best_predictions.values()
        for i in subset.values()
    )
)
maxabs = min(maxabs, 6)

# Initialize figure
fig, axes = plt.subplots(
    nrows=3,
    ncols=3,
    figsize=(15, 15),
    sharex=True,
    sharey=True,
)

# Plot the differences between the target and the best prediction for each layout
axes = axes.T
for ax_col, depth_type in zip(axes, best_predictions):
    for ax, layout in zip(ax_col, best_predictions[depth_type]):
        prediction = best_predictions[depth_type][layout]
        tmp = ax.pcolormesh(
            *grid[:2],
            target - prediction,
            vmin=-maxabs,
            vmax=maxabs,
            cmap="seismic",
        )
        ax.scatter(survey.easting, survey.northing, s=1, alpha=0.2, color="k")
        ax.set_aspect("equal")
        ax.set_title("{} (r2: {:.3f})".format(layout, r2_score(target, prediction)))      

# Hide the last two axes because they are not used
axes[-1][-1].set_visible(False)
axes[-2][-1].set_visible(False)

# Annotate the columns of the figure
for i, depth_type in enumerate(list(best_predictions.keys())):
    axes[i][0].text(
        .5,
        1.1,
        depth_type,
        fontsize="large",
        fontweight="bold",
        horizontalalignment='center',
        transform=axes[i][0].transAxes,
    )

# Add colorbar
fig.subplots_adjust(bottom=0.1, wspace=0.05)
cbar_ax = fig.add_axes([0.15, 0.05, 0.7, 0.02])
fig.colorbar(tmp, cax=cbar_ax, orientation="horizontal", label=field_units)

plt.show()